In [ ]:
import boto3

def list_folders_recursive(bucket_name, prefix=""):
    s3_client = boto3.client(
           's3',
            endpoint_url='http://10.100.100.61:9000',
            aws_access_key_id='minioadmin',
            aws_secret_access_key='minioadmin',
            region_name='us-east-1',
    )

    folders = set()
    continuation_token = None  # Para paginação

    while True:
        list_params = {"Bucket": bucket_name, "Prefix": prefix}
        if continuation_token:
            list_params["ContinuationToken"] = continuation_token

        response = s3_client.list_objects_v2(**list_params)

        if "Contents" in response:
            for obj in response["Contents"]:
                key = obj["Key"]
                parts = key.split("/")[:-1]  
                for i in range(1, len(parts) + 1):
                    folders.add("/".join(parts[:i]) + "/")

        if response.get("IsTruncated"):
            continuation_token = response["NextContinuationToken"]
        else:
            break

    return sorted(folders)

    def filtrar_paths(paths, codigos):
        paths_filtrados = []
        
        for path in paths:
            path_lower = path.lower()  
            
            for codigo in codigos:
                codigo_lower = codigo.lower()
                if codigo_lower in path_lower and path_lower.endswith(".parquet"):
                    paths_filtrados.append(path)
                    break 

        return paths_filtrados

# Exemplo de uso
bucket_name = "bronze"
prefix = ""  

folders = list_folders_recursive(bucket_name, prefix)

print("Pastas encontradas:")
for folder in folders:
    print(folder)


Pastas encontradas:
ibge-sim-sinasc/
ibge-sim-sinasc/ibge_data_raw/
ibge-sim-sinasc/ibge_data_raw/DTB_2021/
ibge-sim-sinasc/ibge_data_raw/GDP_1999_2001/
ibge-sim-sinasc/ibge_data_raw/GDP_2002_2009/
ibge-sim-sinasc/ibge_data_raw/GDP_2010_2019/
ibge-sim-sinasc/ibge_data_raw/ibge-censo-2022/
ibge-sim-sinasc/ibge_data_raw/population_censo_2000/
ibge-sim-sinasc/ibge_data_raw/population_censo_2010/
ibge-sim-sinasc/ibge_data_raw/population_count_1996/
ibge-sim-sinasc/ibge_data_raw/population_count_2007/
ibge-sim-sinasc/ibge_data_raw/population_estimates/
ibge-sim-sinasc/ipea_data_raw/
ibge-sim-sinasc/sim_data_raw/
ibge-sim-sinasc/sinasc_data_raw/
ibge-sim-sinasc/sinasc_data_raw/sinasc-resto/
sia-parquet/
sia-parquet/ACF/
sia-parquet/ACF/ACFGO2401.parquet/
sia-parquet/ACF/ACFGO2402.parquet/
sia-parquet/ACF/ACFGO2403.parquet/
sia-parquet/ACF/ACFGO2404.parquet/
sia-parquet/ACF/ACFGO2405.parquet/
sia-parquet/ACF/ACFGO2407.parquet/
sia-parquet/ACF/ACFGO2408.parquet/
sia-parquet/ACF/ACFGO2409.parqu

In [9]:
import itertools

siglas = ['ACF']
anos = [2021]
estados = ['PB']
meses = [f"{i:02}" for i in range(1, 13)]

print()

['ACFPB2101', 'ACFPB2102', 'ACFPB2103', 'ACFPB2104', 'ACFPB2105', 'ACFPB2106', 'ACFPB2107', 'ACFPB2108', 'ACFPB2109', 'ACFPB2110', 'ACFPB2111', 'ACFPB2112']
